In [ ]:
from bs4 import BeautifulSoup
import urllib2
import requests
import numpy as np
import re
import pandas as pd
from itertools import chain
#from spacy.en import English
#from __future__ import print_function
#import gensim

In [ ]:
def SoupMaker(url):
    opener = urllib2.build_opener()
    opener.addheaders = [('User-agent', 'Mozilla/43.0.3')]
    response = opener.open(url)
    page = response.read()
    return BeautifulSoup(page,'html5lib')

In [ ]:
base = 'http://www.c-span.org/search/?searchtype=Videos&sort=Most+Recent+Airing&seriesid[]=91&sdate=&edate=&searchtype=Videos&sort=Most+Recent+Airing&text=0&seriesid%5B%5D=91&ajax&page='
base_range = xrange(1,57)
base_scrapes = [base+str(a) for a in base_range]
base_soups = pd.Series(base_scrapes).apply(SoupMaker)

In [ ]:
anchors_nested = [a.find('ul',{'class':'video-results'}).findAll('a',{'class':'title'}) for a in base_soups]
anchors = list(chain(*anchors_nested))
urls = [a['href'] for a in anchors]

In [ ]:
req_end = '&action=getTranscript&transcriptType=cc&service-url=%2Fcommon%2Fservices%2FprogramSpeakers.php&progid=440129&appearance-filter=&personSkip=0&ccSkip=0&transcriptSpeaker=&transcriptQuery='
url_series = pd.Series(urls)
req_series = url_series + req_end
text_series = req_series.apply(SoupMaker)
all_tds = [a.findAll('td',{'class':None}) for a in list(text_series)]
all_tds_series = pd.Series(all_tds)

In [ ]:
def parse_cell(cell):
    if cell:
        speaker = cell.find('strong').text
        ft = cell.text
        text = ft[ft.find(speaker)+len(speaker):]
        return (speaker,text)

In [ ]:
def full_transcript(ser):
    new_ser = ser[ser.notnull()]
    transcript = ''
    for i in new_ser.iteritems():
        transcript += i[1].text
    return transcript
    

In [ ]:
all_td_df = all_tds_series.apply(list).apply(pd.Series).T
transcript_series = all_td_df.apply(full_transcript)

In [ ]:
def get_title(x):
    return x[x.rfind('/')+1:]

In [ ]:
titles = url_series.apply(get_title)

# Save output (so we don't have to scrape again...)

In [ ]:
out_df = pd.DataFrame([titles,transcript_series]).T
out_df.columns = ['title','transcript']

In [ ]:
text_series.to_csv('all_text.csv',encoding='utf-8')

In [ ]:
out_df.to_csv('trimmed_columns.csv',encoding='utf-8')

### Starting here is a different way of formulating transcripts. Ended up abandoning based on issues where speakers were never identified. Need to build automated system to detect when a speaker isn't correctly identified and flag for manual review

In [ ]:
candiate_list = ['Bernie Sanders','Hillary Clinton','Donald Trump','Ted Cruz','John Kaisch','Marco Rubio']
re_candidate_test =  "/"+' '.join(candiate_list).replace(' ','|').upper()+"/"

##### Below is a different build transcript that relies on whom the speaker is

In [ ]:
def build_transcript(tuple_list,re_pat):
    speakers = [a[0].upper() for a in tuple_list]
    text = [a[1] for a in tuple_list]
    result = ''
    for i in xrange(0,len(text)):
        if re.search(re_candidate_test,str(speakers[i])):
            result += text[i]
        else:
            pass
    return result